In [1]:
import os
import cv2
import numpy as np
from PIL import Image

In [2]:

# Define paths to videos and images datasets
videos_dir1 = (r"C:\Users\Lenovo\OneDrive\Desktop\Detection\Violence")
videos_dir2 = "C:\\Users\\Lenovo\\OneDrive\\Desktop\\Detection\\NonViolence"
images_dir = "C:\\Users\\Lenovo\\OneDrive\\Desktop\\Detection\\Fire_smoke"
combined_dir = "C:\\Users\\Lenovo\\OneDrive\\Desktop\\Detection\\Combined_Dataset"

# Create the combined directory if it doesn't already exist
if not os.path.exists(combined_dir):
    os.makedirs(combined_dir)


# Load video frames from Violence videos dataset
for filename in os.listdir(videos_dir1):
    video_path1 = os.path.join(videos_dir1, filename)
    cap = cv2.VideoCapture(video_path1)
    if not cap.isOpened():
        print(f"Could not open {video_path1}")
        continue
    frame_count1 = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(min(frame_count1, 10)):  # Stop after 10 frames have been extracted
        ret, frame = cap.read()
        if not ret:
            print(f"Error reading frame {i} from {video_path1}")
            break
        # Convert video frame to an image and save it to the combined directory
        image1 = Image.fromarray(frame)
        image1.save(os.path.join(combined_dir, f"{filename}_{i}.jpg"))

        # Display the current frame
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

            
# Load video frames from NonViolence videos dataset
for filename in os.listdir(videos_dir2):
    video_path2 = os.path.join(videos_dir2, filename)
    cap = cv2.VideoCapture(video_path2)
    if not cap.isOpened():
        print(f"Could not open {video_path2}")
        continue
    frame_count2 = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    for i in range(min(frame_count2, 10)):  # Stop after 10 frames have been extracted
        ret, frame = cap.read()
        if not ret:
            print(f"Error reading frame {i} from {video_path2}")
            break
        # Convert video frame to an image and save it to the combined directory
        image2 = Image.fromarray(frame)
        image2.save(os.path.join(combined_dir, f"{filename}_{i}.jpg"))

        # Display the current frame
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break



for filename in os.listdir(images_dir):
    image_path = os.path.join(images_dir, filename)
    image = Image.open(image_path)
    # Save the image to the combined directory
    image.save(os.path.join(combined_dir, filename))

    # Display the current image
    image.show()

            
            
            
# Wait for a key press to exit
cv2.waitKey(0)

# Destroy all windows
cv2.destroyAllWindows()


In [ ]:

# Set the path to the combined dataset
Combined_Dataset = "C:\\Users\\Lenovo\\OneDrive\\Desktop\\Detection\\Combined_Dataset"




In [3]:
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [30]:
def create_dataset():
   
    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []
    
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):
        
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)

            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:

                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)  
    
    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

In [31]:
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: walking


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Combined_Dataset\\walking'

In [ ]:
def frames_extraction(video_path):

    # Declare a list to store video frames.
    frames_list = []
    
    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)
   
    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading the frame from the video. 
        success, frame = video_reader.read() 

        # Check if Video frame is not successfully read then break the loop
        if not success:
            break

        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
        
        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)
    
    # Release the VideoCapture object. 
    video_reader.release()

    # Return the frames list.
    return frames_list